In [ ]:
import calendar
import json
from collections import Counter
from datetime import date
from pprint import pprint
import os
from pymongo import MongoClient
import pandas as pd

In [ ]:
with open(os.path.expanduser('~/.mongo/mongo.json'),'r') as config:
    mongo_uri = json.load(config)['mongo_uri']
client = MongoClient(mongo_uri)
db = client.forethought

organization_id = '432'
start = '2017-04-01T00:00:00Z'
end = '2021-06-07T00:00:00Z'
mongo_org_collection = db[str(organization_id)]

In [ ]:
def get_dataset():
    tag = 'shipping/carrier_calls__contact_information_needed'
    macro = '360104523652'
    tickets = mongo_org_collection.find({
        'doc_type': 'ticket',
        "is_deleted": False,
        'original_item.via.channel': {'$in':['web','email', "api"]},
        'original_item.created_at': {'$gte': start, '$lte': end},
        "original_item.via.source.rel": {"$ne": 'voicemail'}
    })
    
    file_path = os.path.expanduser('~/Downloads/sdso_macro.json')
    with open(file_path, 'w') as f:
        count = 0
        for ticket in tickets:
            i = ticket["original_item"]
            label = macro if tag in i['tags'] else 'no macro applied'
            j = {
                "ticket_id": 'id', 
                "subject": i["subject"],
                "description": i["description"],
                "created_at": i["created_at"],
                "label": label
                }
                
            json.dump(j, f)
            _ = f.write('\n')
            count += 1
            if count % 100 == 0:
                print(count)

    print("total: ", count)


get_dataset()